In [1]:
# 读入表格 res_Albumin.csv

import pandas as pd
import numpy as np
import os
import sys
import math

result_prefix = "./result/statistics_r/res/time"
df_origin = pd.read_csv(f'{result_prefix}/res_Albumin.csv',index_col=0)


# Format the result of stage2

In [2]:
df_origin

,diet,stage,round1,round2,round3,p,p_nonlinear,d1,d2,d3
1,water,0,12.76,13.72,-1.0,0.1538,0.0791,down,up,-1
2,protein,0,52.23,62.02,-1.0,0.0056,0.0028,down,up,-1
3,fat,0,5.88,7.36,-1.0,0.0001,0.0000,down,up,-1
4,carbohydrate,0,194.82,296.37,-1.0,0.0011,0.0007,down,up,-1
5,Calories,0,150.10,199.18,-1.0,0.0009,0.0005,down,up,-1
...,...,...,...,...,...,...,...,...,...,...
100,riboflavin,3,-1.00,-1.00,-1.0,0.0003,0.0006,-1,-1,-1
101,nicotinic,3,11.57,20.03,-1.0,0.0000,0.0000,down,up,-1
102,ascorbic,3,79.27,-1.00,-1.0,0.0020,0.0961,down,-1,-1
103,dpi,3,8.75,12.83,-1.0,0.0000,0.0002,down,up,-1


In [3]:


#---------------------------------
# 增加一列，边界boundary，字符串形式
units_list ={'water':'g/d', 'protein':'g/d', 'fat':'g/d', 'carbohydrate':'g/d', 'Calories':'kcal/d', 'df':'g/d', 
'k':'mg/d', 'na':'mg/d', 'mg':'mg/d', 'ca':'mg/d', 'p':'mg/d', 'fe':'mg/d', 'zn':'mg/d', 'cu':'mg/d', 'mn':'mg/d', 'se':'mg/d', 'retinol':'ug/d', 'vitaminA':'ugRAE/d', 'carotene':'ug/d', 
'vitaminE':'mg/d', 'thiamine':'mg/d', 'riboflavin':'mg/d', 'nicotinic':'mg/d', 'ascorbic':'mg/d', 'dpi':'g/kg/d','dei':'kcal/kg/d'}
names_list = {'water':'Water', 'protein':'Protain', 'fat':'Fat', 'carbohydrate':'Carbohydrate', 'Calories':'Calories', 'df':'Df', 'k':'K', 'na':'Na', 'mg':'Mg', 
              'ca':'Ca', 'p':'P', 'fe':'Fe', 'zn':'Zn', 'cu':'Cu', 'mn':'Mn', 
              'se':'Se', 'retinol':'Retinol', 'vitaminA':'Vitamin A', 'carotene':'Carotene', 'vitaminE':'Vitamin E', 'thiamine':'Thiamine', 'riboflavin':'Riboflavin', 
              'nicotinic':'Nicotinic', 'ascorbic':'Ascorbic', 'dpi':'DPI', 'dei':'DEI'}


div_dict = {'water':100,'protein':1,'fat':10,
                'carbohydrate':1,'Calories':10,'df':1,'k':10,'na':100,
                'mg':10,'ca':100,'p':10,'fe':1,'zn':1,'cu':1,
                'mn':1,'se':1,'retinol':10,'vitaminA':100,
                'carotene':100,'vitaminE':1,'thiamine':1,'riboflavin':1,
                'nicotinic':1,'ascorbic':1,'dpi':0.1,'dei':1,'hsCRP':1}

# 修改units_list
for key in units_list.keys():
    cur_unit = units_list[key]
    if(div_dict[key] != 1):
        units_list[key] =  f'{div_dict[key]}{cur_unit}'

# 遍历每一行，计算边界
for index, row in df_origin.iterrows():
    # print(index)
    # 如果round1 = -1，boundary = ‘-’
    # 如果round2 = -1，boundary = (round1, -)
    # 如果round3 = -1，boundary = (round1, round2)
    # 如果round3 != -1，boundary = (round1, round2) and (round3, -)
    # 保留两位小数
    if(row['round1'] == -1):
        boundary = '-'
    elif(row['round2'] == -1):
        boundary = '(' + f'{row["round1"]:.2f}' + ', -)'
    elif(row['round3'] == -1):
        boundary = '(' + f'{row["round1"]:.2f}' + ', ' + f'{row["round2"]:.2f}' + ')'
    else:
        boundary = '(' + f'{row["round1"]:.2f}' + ', ' + f'{row["round2"]:.2f}' + ') or (' + f'{row["round3"]:.2f}' + ', -)'
    df_origin.loc[index, 'boundary'] = boundary


# 增加一列p_str,对于p值，则 为*，如果p<0.001,则为***，如果p<0.01,则为**，如果p<0.05,否则为空,用pd方法实现
df_origin['p_str'] = df_origin['p'].apply(lambda x: '***' if x < 0.001 else ('**' if x < 0.01 else ('*' if x < 0.05 else '')))
# p_linear_str, 映射p_linear
df_origin['p_nonlinear_str'] = df_origin['p_nonlinear'].apply(lambda x: '***' if x < 0.001 else ('**' if x < 0.01 else ('*' if x < 0.05 else '')))
df_origin['unit'] = [units_list[i] for i in df_origin['diet']]

df_origin.to_csv(f'{result_prefix}/res_Albumin_new.csv')
df_origin.to_excel(f'{result_prefix}/res_Albumin_new.xlsx')

# --------------------------------------------
# copy表格，只保留列diet，stage，round1，round2，round3，p_str，p_linear_str
df_new = df_origin[['diet','unit','stage','round1','round2','round3','p_str','p_nonlinear_str']]
# 遍历每一行，round1，round2，round3，如果为-1，则置为 -
for index, row in df_new.iterrows():
    # 两位小数
    df_new.loc[index, 'round1'] = f'{row["round1"]:.2f}'
    df_new.loc[index, 'round2'] = f'{row["round2"]:.2f}'
    df_new.loc[index, 'round3'] = f'{row["round3"]:.2f}'
    if(int(row['round1']) == -1):
        df_new.loc[index, 'round1'] = '-'
    if(int(row['round2']) == -1):
        df_new.loc[index, 'round2'] = '-'
    if(int(row['round3']) == -1):
        df_new.loc[index, 'round3'] = '-'
df_new['diet'] = df_new['diet'].apply(lambda x : x.upper() if x == 'dpi' or x == 'dei' else x.capitalize())

df_new.to_excel(f'{result_prefix}/res_Albumin_new_str.xlsx')




df_subs = []
df_diet = df_new[['diet','unit']]
df_new = df_new.drop('diet',axis=1)
df_new = df_new.drop('unit',axis=1)
df_new = df_new.groupby('stage')

for index, df_sub in df_new:
    # print(index)
    # print(df_sub)
    df_sub = df_sub.drop('stage',axis=1)
    df_sub.columns = [str(index) + '_' + i for i in df_sub.columns]
    df_sub.index = range(len(df_sub))
    df_subs.append(df_sub)
df_diet = df_diet[:len(df_sub)]
df_diet.index = range(len(df_diet))
df_diet = pd.concat([df_diet] + df_subs, axis=1)


df_diet.to_excel(f'{result_prefix}/res_Albumin_finally.xlsx')


# ------------------------------------------
# 只保留列diet，stage，boundary，p_str，p_linear_str
df_new = df_origin[['diet','unit','stage','boundary','p_str','p_nonlinear_str']]
# 先去除列diet，保存到其他变量中
df_diet = df_new[['diet','unit']]
df_new = df_new.drop('diet',axis=1)
df_new = df_new.drop('unit',axis=1)
# 按照stage分组，作为新的列，如boundary0，p_str0，p_linear_str0，boundary1，p_str1，p_linear_str1，boundary2，p_str2，p_linear_str2
df_new = df_new.groupby('stage')
df_subs = []
for index, df_sub in df_new:
    # print(index)
    # print(df_sub)
    df_sub = df_sub.drop('stage',axis=1)
    df_sub.columns = [i+'_'+str(index) for i in df_sub.columns]
    df_sub.index = range(len(df_sub))
    df_subs.append(df_sub)
# df_diet的前len(df_sub)行
df_diet = df_diet[:len(df_sub)]
df_diet.index = range(len(df_diet))
# 横向合并df_new的各个分组
df_new = pd.concat([df_diet] + df_subs, axis=1)
print(df_new.shape)
# 保存表格
df_new.to_csv(f'{result_prefix}/res_Albumin_new_cut.csv')

(26, 14)


/tmp/ipykernel_2784/849565895.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12.76' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_new.loc[index, 'round1'] = f'{row["round1"]:.2f}'
/tmp/ipykernel_2784/849565895.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '13.72' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_new.loc[index, 'round2'] = f'{row["round2"]:.2f}'
/tmp/ipykernel_2784/849565895.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1.00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_new.loc[index, 'round3'] = f'{row["round3"]:.2f}'
/tmp/ipykernel_2784/849565895.py:68: SettingWithC

In [4]:
# 按照stage分组，作为新的列
df_origin_clone = df_origin.copy()
df_origin_clone['p_str'] = df_origin_clone['p'].apply(lambda x: '<0.001' if x < 0.001 else '<0.01' if x < 0.01 else f'{x:.2f}')
df_origin_clone['p_nonlinear_str'] = df_origin_clone['p_nonlinear'].apply(lambda x: '<0.001' if x < 0.001 else '<0.01' if x < 0.01 else f'{x:.2f}')
# 遍历每一行，如果round值为-1，则变为-
df_origin_clone['round1'] = df_origin_clone['round1'].apply(lambda x: '-' if x == -1 else f'{x:.2f}')
df_origin_clone['round2'] = df_origin_clone['round2'].apply(lambda x: '-' if x == -1 else f'{x:.2f}')
df_origin_clone['round3'] = df_origin_clone['round3'].apply(lambda x: '-' if x == -1 else f'{x:.2f}')
df_new = df_origin_clone.groupby('stage')
df_subs = []
for index, df_sub in df_new:
    df_sub = df_sub.drop(columns=['stage'],axis=1)
    df_sub.columns = [f'{x}_{index}' for x in df_sub.columns]
    df_sub.index = range(len(df_sub))
    df_subs.append(df_sub)
df_diet = df_origin_clone[:len(df_subs[0])][['diet','unit']]
# df_diet = df_diet.drop(columns=['stage'],axis=1)
df_diet.index = range(len(df_diet))
df_diet_new = pd.concat([df_diet] + df_subs,axis=1)
# print(df_diet_new)

# print to latex
for index, row in df_diet_new.iterrows():
    cur_diet_label = names_list[row['diet']]
    print(f'{cur_diet_label} & {row["unit"]} & {row["round1_3"]} & {row["round2_3"]} & {row["round3_3"]} & {row["p_str_3"]} & {row["p_nonlinear_str_3"]} & {row["round1_0"]} & {row["round2_0"]} & {row["round3_0"]} & {row["p_str_0"]} & {row["p_nonlinear_str_0"]} & {row["round1_1"]} & {row["round2_1"]} & {row["round3_1"]} & {row["p_str_1"]} & {row["p_nonlinear_str_1"]} & {row["round1_2"]} & {row["round2_2"]} & {row["round3_2"]} & {row["p_str_2"]} & {row["p_nonlinear_str_2"]} \\\\')

Water & 100g/d & 14.11 & - & - & <0.01 & 0.10 & 12.76 & 13.72 & - & 0.15 & 0.08 & 13.79 & - & - & 0.81 & 0.84 & 14.45 & - & - & 0.02 & 0.29 \\
Protain & g/d & 50.29 & 74.76 & - & <0.001 & <0.001 & 52.23 & 62.02 & - & <0.01 & <0.01 & 46.99 & 48.48 & - & <0.01 & 0.01 & 51.29 & - & - & <0.01 & 0.21 \\
Fat & 10g/d & 5.91 & 7.72 & - & <0.001 & <0.001 & 5.88 & 7.36 & - & <0.001 & <0.001 & 5.78 & 5.84 & - & <0.001 & <0.01 & 6.04 & - & - & <0.001 & 0.49 \\
Carbohydrate & g/d & 206.26 & 382.04 & - & <0.001 & 0.02 & 194.82 & 296.37 & - & <0.01 & <0.001 & 201.57 & - & - & 0.22 & 0.81 & 216.48 & - & - & 0.02 & 0.63 \\
Calories & 10kcal/d & 152.47 & 238.33 & - & <0.001 & <0.001 & 150.10 & 199.18 & - & <0.001 & <0.001 & 148.13 & 197.21 & - & 0.06 & 0.10 & 156.05 & - & - & <0.001 & 0.42 \\
Df & g/d & 9.73 & 15.45 & - & <0.001 & <0.001 & 9.44 & 14.60 & - & <0.001 & <0.001 & 9.20 & 9.90 & 13.56 & 0.04 & 0.15 & 10.17 & 16.00 & - & <0.001 & <0.01 \\
K & 10mg/d & 144.09 & 235.32 & - & <0.001 & <0.001 & 14

# Format the result of stage1

In [5]:
# 读入表格 res_Albumin.csv

import pandas as pd
import numpy as np
import os
import sys
import math

df_cox_origin = pd.read_csv(f'{result_prefix}/res_cox_Albumin.csv',index_col=0)

In [6]:
df_cox_origin

,col,stage,c,c1,c2,z,p,951,952
1,water,0,-0.017090,0.983056,0.028955,-0.590211,0.555049,0.928820,1.040458
2,protein,0,-0.018919,0.981259,0.011177,-1.692628,0.090526,0.959996,1.002993
3,fat,0,-0.031955,0.968550,0.089832,-0.355721,0.722050,0.812189,1.155013
4,carbohydrate,0,-0.005346,0.994669,0.002701,-1.979511,0.047759,0.989418,0.999947
5,Calories,0,-0.007695,0.992335,0.004283,-1.796774,0.072371,0.984040,1.000699
...,...,...,...,...,...,...,...,...,...
100,riboflavin,3,0.018448,1.018619,0.013498,1.366752,0.171703,0.992025,1.045926
101,nicotinic,3,-0.084912,0.918594,0.020971,-4.049050,0.000051,0.881603,0.957136
102,ascorbic,3,-0.005560,0.994456,0.001876,-2.963585,0.003041,0.990806,0.998119
103,dpi,3,-0.163462,0.849199,0.039247,-4.164909,0.000031,0.786325,0.917100


In [7]:
# 按照stage分组，作为新的列，如c_0,c_1_0,c_2_0等
df_cox_origin['p_str'] = df_cox_origin['p'].apply(lambda x: '<0.001' if x < 0.001 else ('<0.01' if x < 0.01 else f'{x:.2f}'))
df_cox_new = df_cox_origin.groupby('stage')
df_subs = []
for index, df_sub in df_cox_new:
    # print(index)
    # print(df_sub)
    df_sub = df_sub.drop('stage',axis=1)
    df_sub.columns = [i+'_'+str(index) for i in df_sub.columns]
    df_sub.index = range(len(df_sub))
    df_subs.append(df_sub)
# df_diet的前len(df_sub)行
df_diet = df_diet[:len(df_sub)]
df_diet.index = range(len(df_diet))
# 横向合并df_new的各个分组
df_cox_new = pd.concat([df_diet] + df_subs, axis=1)
print(df_cox_new.shape)
# print(df_cox_new)

# out put table for latex , col, c1, f'{951}-{952}'(保留两位小数)，p
for index, row in df_cox_new.iterrows():
    cur_diet = row['diet']
    print(f"{names_list[cur_diet]} & {units_list[cur_diet]} & {row['c1_3']:.2f} & {row['951_3']:.2f}-{row['952_3']:.2f} & {row['p_str_3']} & {row['c1_0']:.2f} & {row['951_0']:.2f}-{row['952_0']:.2f} & {row['p_str_0']} & {row['c1_1']:.2f} & {row['951_1']:.2f}-{row['952_1']:.2f} & {row['p_str_1']} & {row['c1_2']:.2f} & {row['951_2']:.2f}-{row['952_2']:.2f} & {row['p_str_2']} \\\\")


(26, 38)
Water & 100g/d & 0.95 & 0.92-0.99 & <0.01 & 0.98 & 0.93-1.04 & 0.56 & 0.97 & 0.91-1.03 & 0.34 & 0.86 & 0.77-0.95 & <0.01 \\
Protain & g/d & 0.97 & 0.96-0.98 & <0.001 & 0.98 & 0.96-1.00 & 0.09 & 0.98 & 0.96-1.01 & 0.15 & 0.94 & 0.91-0.97 & <0.001 \\
Fat & 10g/d & 0.82 & 0.74-0.92 & <0.001 & 0.97 & 0.81-1.16 & 0.72 & 0.84 & 0.71-1.01 & 0.06 & 0.58 & 0.44-0.76 & <0.001 \\
Carbohydrate & g/d & 0.99 & 0.99-1.00 & <0.001 & 0.99 & 0.99-1.00 & 0.05 & 0.99 & 0.99-1.00 & 0.04 & 0.99 & 0.98-1.00 & <0.01 \\
Calories & 10kcal/d & 0.99 & 0.98-0.99 & <0.001 & 0.99 & 0.98-1.00 & 0.07 & 0.99 & 0.99-1.00 & 0.09 & 0.98 & 0.96-0.99 & <0.001 \\
Df & g/d & 0.90 & 0.86-0.95 & <0.001 & 0.93 & 0.86-1.00 & 0.06 & 0.92 & 0.85-1.00 & 0.06 & 0.80 & 0.70-0.90 & <0.001 \\
K & 10mg/d & 0.99 & 0.99-1.00 & <0.001 & 0.99 & 0.99-1.00 & 0.08 & 1.00 & 0.99-1.01 & 0.64 & 0.98 & 0.97-0.99 & <0.001 \\
Na & 100mg/d & 0.97 & 0.95-0.99 & <0.01 & 1.00 & 0.97-1.03 & 0.76 & 0.97 & 0.94-1.00 & 0.09 & 0.90 & 0.86-0.95 & <0.0

In [8]:
# 获取col,c1,951,952,df_origin的p
df_cox_final = df_cox_origin[['col','stage','c1','951','952']] 
# df_cox_final['p'] = df_origin['p']
df_cox_final['ci'] = df_cox_origin.apply(lambda row: f"{row['951']:.2f}-{row['952']:.2f}",axis=1)
df_cox_final['c1'] = df_cox_final['c1'].apply(lambda x : f'{x:.2f}')
df_cox_final['p'] = df_cox_origin['p'].astype(float)
df_cox_final['p'] = df_cox_final['p'].apply(lambda x: '***' if x < 0.001 else ('**' if x < 0.01 else ('*' if x < 0.05 else '')))
df_cox_final = df_cox_final[['col','stage','c1','ci','p']]
# col一列，替换成首字母大写的
df_cox_final['col'] = df_cox_final['col'].apply(lambda x : x.upper() if x == 'dpi' or x == 'dei' else x.capitalize())
df_cox_final.to_excel(f'{result_prefix}/res_cox_Albumin_final.xlsx')

/tmp/ipykernel_2784/905169952.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox_final['ci'] = df_cox_origin.apply(lambda row: f"{row['951']:.2f}-{row['952']:.2f}",axis=1)
/tmp/ipykernel_2784/905169952.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox_final['c1'] = df_cox_final['c1'].apply(lambda x : f'{x:.2f}')
/tmp/ipykernel_2784/905169952.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste